In [39]:
import random
from collections import defaultdict
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from task.multi_task import filter_tasks
from utils.stim_io import HvMImageLoader, HvMMetaData, HvMImageMapper, _subpath_after
import shutil

In [65]:
hvm_dir = '/Users/markbai/PycharmProjects/RNN_NatAbs/data/original'
OUTPUT_CSV = "../resources/interdms_position_identity_trials.csv"
IMAGES_DIR = Path('images')
BG_FP = IMAGES_DIR / 'gray_background.png'

SESSIONS = [1, 2, 3, 4, 5]
TRIALS_PER_SESSION = 20
FRAMES_PER_TRIAL = 6
RANDOM_SEED = 2025
GRID_SIZE = 3
ACTION_MAP = {
    0: 'none',  # no action
    1: 'x',  # match
    2: 'b',  # non-match
}

random.seed(RANDOM_SEED)
meta = HvMMetaData(hvm_dir)
img_loader = HvMImageLoader(
    root_dir=hvm_dir,
    metadata=meta,
    preload_images=False,
)
img_loader.prepare_for_tasks(GRID_SIZE)
img_loader.df

removed 0 rows from the original df
found 5760 local images after filtering


,_id,bg_id,category,filename,id,obj,rxy,rxy_semantic,rxz,rxz_semantic,...,size,ty,tz,var,category_id,obj_id,file_exists,cat_1b,id_1b,pos_1b
0,01c0425e982e4edc276fe5373dbf8e5e49bd9a16_V0,01c0425e982e4edc276fe5373dbf8e5e49bd9a16,Fruits,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,f1ae7c2160c6cbb309b940a0dc127a8aa1060e5d,Apple_Fruit_obj,0.000156,-0.000,0.000067,0.000,...,256.0,0.000,0.000,0,5,0,True,6,1,5
1,1ab7a705f57d772a0203141ad42742e874f42099_V0,1ab7a705f57d772a0203141ad42742e874f42099,Fruits,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,691384450355c07ad084613a08a018ae8e1eebb2,Apple_Fruit_obj,0.000156,-0.000,0.000067,0.000,...,256.0,0.000,0.000,0,5,0,True,6,1,5
2,813820d67a6fe96bc9167070d85dd6258e9b0b47_V0,813820d67a6fe96bc9167070d85dd6258e9b0b47,Fruits,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,fe9cd029e43ebdeeb509b61db1573a8286d06e6a,Apple_Fruit_obj,0.000156,-0.000,0.000067,0.000,...,256.0,0.000,0.000,0,5,0,True,6,1,5
3,838fc10ae4d5aecb22595ca7e1aba67fd0fd9c32_V0,838fc10ae4d5aecb22595ca7e1aba67fd0fd9c32,Fruits,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,c766632574f6d86f0a048f2e5817826c1888699b,Apple_Fruit_obj,0.000156,-0.000,0.000067,0.000,...,256.0,0.000,0.000,0,5,0,True,6,1,5
4,8c1ccb3fbdd6cfb27854f387225380a16774cddc_V0,8c1ccb3fbdd6cfb27854f387225380a16774cddc,Fruits,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,7edd311b0ef7a9e61c62c6cbc2b3cfc2b5b6ea9d,Apple_Fruit_obj,0.000156,-0.000,0.000067,0.000,...,256.0,0.000,0.000,0,5,0,True,6,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5755,566bd813b77921c9376c49b62e34774a488c7db1_V6,566bd813b77921c9376c49b62e34774a488c7db1,Cars,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,f929a349245a4e77a8c6a912ce25caa0e66f1dc3,z3,0.554070,139.824,-0.870254,-78.281,...,256.0,0.125,-0.770,6,2,7,True,3,8,8
5756,1ceb599d23a380aa296079e6c0b4606e942e75f8_V6,1ceb599d23a380aa296079e6c0b4606e942e75f8,Cars,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,81b744f089e25237e73e7e0005cbdb1e3d99f214,z3,0.336224,120.229,-0.951715,-85.608,...,256.0,-0.194,-0.077,6,2,7,True,3,8,5
5757,4c520473a9e43a98f1785cfe134f7a5dd9759d68_V6,4c520473a9e43a98f1785cfe134f7a5dd9759d68,Cars,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,de243d91c7d8d5f04c6d28ed955b883b6299eba7,z3,0.715683,154.361,-0.956874,-86.072,...,256.0,-0.283,0.244,6,2,7,True,3,8,6
5758,0bbb182f4a889bc4688daf39635e2c9f8309298e_V6,0bbb182f4a889bc4688daf39635e2c9f8309298e,Cars,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,ddb6642885da547e2e7341335927e9ea44e0dd6c,z3,-0.577883,38.006,-0.965879,-86.882,...,256.0,-0.014,0.402,6,2,7,True,3,8,2


In [66]:
rows = list()
df = img_loader.df
# find 50 stims, each with 2 locations
for catid, positions in img_loader._task_cache.catid_to_positions.items():
    cat, obj = catid
    chosen_pos = random.choices(positions, k=2)
    subset = df[
        (df["category_id"] == cat) &
        (df["obj_id"] == obj) &
        (df["pos_1b"].isin(chosen_pos))
        ]
    rows.append(subset)

df_subset = pd.concat(rows, ignore_index=True)
df_subset = df_subset.sample(n=min(100, len(df_subset)), random_state=0)
img_loader.df = df_subset
df_subset

,_id,bg_id,category,filename,id,obj,rxy,rxy_semantic,rxz,rxz_semantic,...,size,ty,tz,var,category_id,obj_id,file_exists,cat_1b,id_1b,pos_1b
879,4ad6f88d75350896b774c53f7dc458017e7c1a00_V0,4ad6f88d75350896b774c53f7dc458017e7c1a00,Tables,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,4bef068b1301b9176ced85dd742b1d07fa35c5ce,_44,0.000156,-0.000000,0.000067,0.000000,...,256.0,0.000,0.000,0,7,7,True,8,8,5
496,cac49056c511781d08ee84eba0e3f5532b4702af_V6,cac49056c511781d08ee84eba0e3f5532b4702af,Chairs,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,e16ab6522d45ee4f454aa8506462532770398c14,_008,0.993252,89.328000,0.715704,64.368000,...,256.0,-0.166,-0.158,6,3,2,True,4,3,5
14,cc284e996ab46fa3570711bec313537e77d9f786_V0,cc284e996ab46fa3570711bec313537e77d9f786,Planes,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,9d89aedf86af6292d4d6fe3774a51c92fb715c4e,_19_flyingBoat,0.000156,90.000000,0.000067,0.000000,...,256.0,0.000,0.000,0,6,2,True,7,3,5
546,4c381cd8ab9f43ee9dbc89de210bc4f5a28eb8f0_V6,4c381cd8ab9f43ee9dbc89de210bc4f5a28eb8f0,Chairs,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,62afaf616a2d1b597a38ae0b8cae24c8532a43ee,_010,0.953018,85.709000,-0.802368,-72.175000,...,256.0,-0.591,0.156,6,3,3,True,4,4,6
55,e91ce8e221b8f9a3d9f4a8743e8fb8871308ab95_V6,e91ce8e221b8f9a3d9f4a8743e8fb8871308ab95,Cars,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,d642b115481942c1f08c95ea8296f4e2613173aa,alfa155,-0.235367,68.815000,-0.111724,-10.055000,...,256.0,-0.254,-1.106,6,2,1,True,3,2,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,314ca82cab5c62871f06597cb1974f044479064d_V3,314ca82cab5c62871f06597cb1974f044479064d,Tables,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,59aa962d4f503134849cacb515c8da9ff7e7b3a6,_10,0.140191,12.596000,0.420802,37.843000,...,256.0,0.010,0.290,3,7,1,True,8,2,5
662,664cc28563bd597498a3b90b62f26abf87bc46c6_V6,664cc28563bd597498a3b90b62f26abf87bc46c6,Cars,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,df09a28415cc4f030ef7cbb6333b3781d0734eec,clio,0.325707,119.283000,0.918228,82.584000,...,256.0,-0.244,-0.011,6,2,6,True,3,7,6
692,6add2175d89b80848de5dad3b51cb19630c49300_V6,6add2175d89b80848de5dad3b51cb19630c49300,Fruits,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,fbb2ae70336a38b1df830e02c27e3e0949c4a6b3,Apricot_obj,-0.246486,-22.185191,0.039144,3.514799,...,256.0,-0.053,0.762,6,5,1,True,6,2,2
236,ce7eb3b7974ef60e4c90c791392d7c60f11d8be6_V3,ce7eb3b7974ef60e4c90c791392d7c60f11d8be6,Faces,/Users/markbai/PycharmProjects/RNN_NatAbs/data...,341485d9ecccef8f19ed9595c8ddd435477660c6,face0002,-0.258013,-23.222000,0.415976,37.409000,...,256.0,-0.199,-0.510,3,4,1,True,5,2,8


In [67]:
dataloaders_dict = filter_tasks([20])
datasets = dict()
for task_name, (DatasetClass, kwargs) in dataloaders_dict.items():
    dataset_kwargs = kwargs.copy()
    dataset_kwargs.update({
        'hvm_loader': img_loader,
        'std': 0.0,
        'pad_to': 6,
        'dataset_size': len(SESSIONS) * TRIALS_PER_SESSION,
    })
    # Instantiate the dataset
    tmp_task = DatasetClass(**dataset_kwargs)
    tmp_task.reset()
    datasets[task_name] = tmp_task
datasets

{'interdms_ABAB_position_identity': <task.dms.InterDMSDataset at 0x1828e54f0>}

In [86]:
def trial_to_row(ds, emb, action, session_id):
    zero_mask = torch.all(torch.isclose(
        emb,
        torch.tensor(0.0, dtype=emb.dtype)
    ), dim=1)
    nonzero_idx = torch.nonzero(~zero_mask).squeeze(1)

    subset = emb[nonzero_idx]
    img_mapper = HvMImageMapper(ds)
    files, decode_tuples = img_mapper._batch_decode_and_find(subset)
    fp_list = list()
    for i in range(FRAMES_PER_TRIAL):
        if i in nonzero_idx:
            fp = files.pop(0)
            fp = Path(fp)
            fp = _subpath_after(
                fp, segment='HvM_with_discfade'
            )
            fp = IMAGES_DIR / fp
        else:
            fp = BG_FP
        fp_list.append(str(fp))

    row = {
        'session': session_id,
    }
    for i, (a, fp) in enumerate(zip(action, fp_list)):
        if i != 0:  # no action on first frame
            row[f'act{i + 1}'] = ACTION_MAP[a.item()]

        row[f'stim{i + 1}'] = fp
    return row


def build_csv_from_dataset(dataset, out_csv: str = OUTPUT_CSV) -> pd.DataFrame:
    rows = []
    session_id = 1
    for i, (emb, action, task_index) in enumerate(dataset):
        if i % TRIALS_PER_SESSION == 0 and i > 0:
            session_id += 1
        rows.append(trial_to_row(dataset, emb, action, session_id))
    stim_cols = [f"stim{i}" for i in range(1, FRAMES_PER_TRIAL + 1)]
    act_cols = [f"act{i}" for i in range(2, FRAMES_PER_TRIAL + 1)]
    df = pd.DataFrame(rows, columns=["session"] + stim_cols + act_cols)
    df.to_csv(out_csv, index=False)
    print(f"Saved: {out_csv} (rows={len(df)})")
    return df


def move_images_to_local_folder(df: pd.DataFrame, images_dir: Path = IMAGES_DIR) -> pd.DataFrame:
    project_dir = Path.cwd().parent
    images_dir = project_dir / 'resources'/ images_dir
    shutil.rmtree(images_dir/'Variation00_20110203', True)
    shutil.rmtree(images_dir/'Variation03_20110128', True)
    shutil.rmtree(images_dir/'Variation06_20110131', True)
    
    fps = df['filename'].unique()
    for fp in fps:
        new_fp = _subpath_after(
            fp, segment='HvM_with_discfade'
        )
        new_fp = images_dir / new_fp
        new_fp.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(fp, new_fp.absolute())
    return


move_images_to_local_folder(df_subset, IMAGES_DIR)
inter_dms = datasets["interdms_ABAB_position_identity"]
out_df = build_csv_from_dataset(inter_dms, out_csv=OUTPUT_CSV)

Saved: ../resources/interdms_position_identity_trials.csv (rows=100)


In [83]:
project_dir = Path.cwd().parent
images_dir = project_dir / 'resources'/ images_dir
shutil.rmtree(images_dir/'Variation00_20110203')